In [ ]:
import numpy as np

from util.zonotope import Zonotope
from util.constrained_zonotope import ConstrainedZonotope

In [ ]:
def make_uncertain_ReLU_NN(NN_spec, dim_in, dim_out, wt_lo, wt_hi):
    """
    Generate biases and uncertain weights for a neural network given the NN_spec, a list with 
    length n_depth where each entry is the width of that layer. The input dimension is dim_in,
    and the output dimension is dim_out. The weights are interval matrices with center (wt_hi 
    + wt_lo) * 0.5 and random delta from 0 to (wt_hi - wt_lo) * 0.5.
    
    The "n_depth" scalar above is specifically the number of HIDDEN layers. The output of this
    function creates additional input and output weight/bias matrices. A network with no 
    hidden layers has an integer value for NN_spec, which is the size of the output of the
    first layer.
    
    The output is a list of length (n__layers + 1).
    """
    # Add input and output layer to NN spec
    NN_wt_mat_sizes = [dim_in]
    for layer in NN_spec:
        NN_wt_mat_sizes.append(layer)
    NN_wt_mat_sizes.append(dim_out)
    
    # Initialize matrices for the NN
    n_layers = len(NN_wt_mat_sizes)
    NN_weights_Inf = []
    NN_weights_Sup = []
    NN_biases = []
    
    for i in range(n_layers - 1):
        # Make random uncertain weights
        Mcenter = np.ones((NN_wt_mat_sizes[i + 1], NN_wt_mat_sizes[i])) * (wt_hi + wt_lo) * 0.5
        Mdelta = np.random.rand(NN_wt_mat_sizes[i + 1], NN_wt_mat_sizes[i]) * (wt_hi - wt_lo) * 0.5
        NN_weights_Inf.append(Mcenter - Mdelta)
        NN_weights_Sup.append(Mcenter + Mdelta)
        
        # Make random biases
        NN_biases.append(np.random.rand(NN_wt_mat_sizes[i + 1], 1) * (wt_hi - wt_lo) + wt_lo)
    
    return NN_weights_Inf, NN_weights_Sup, NN_biases

In [ ]:
def intmat_mul(A_Inf, A_Sup, B_Inf, B_Sup):
    """
    Perform matrix multiplication A * B = C for two uncertain matrices A and B. Equation from 
    Eq. 2 of S.P. Adam et al. "Solving the linear interval tolerance problem for weight 
    initialization of neural networks".
    """
    # Compute all multiplication possibilities
    InfInf = A_Inf @ B_Inf
    SupSup = A_Sup @ B_Sup
    SupInf = A_Sup @ B_Inf
    InfSup = A_Inf @ B_Sup
    
    # Preallocat the output
    C_Inf = np.zeros((InfInf.shape))
    C_Sup = np.zeros((InfInf.shape))
    
    # Compute C element-by-element
    for i in range(InfInf.shape[0]):
        for j in range(InfInf.shape[1]):
            ele_lst = [InfInf[i][j], SupSup[i][j], SupInf[i][j], InfSup[i][j]]  # List of possible elements
            C_Inf[i][j] = min(ele_lst)
            C_Sup[i][j] = max(ele_lst)
    
    return C_Inf, C_Sup

In [ ]:
def uncertain_linear_layer_con_zono(Z_in_Inf, Z_in_Sup, W_Inf, W_Sup, b):
    """
    Multiply and add an uncertain constrained zonotope with infimum Z_in_Inf and supremum 
    Z_in_Sup, uncetain weights with infimum W_Inf and supremum W_Sup, and biases b in a neural
    network. Returns the uncertain output constrained zonotope Z_out.
    """
    # Preallocate the output
    Z_out_Inf = []
    Z_out_Sup = []
    
    # Get the number of input zonotopes
    n_in = len(Z_in_Inf)
    
    for i in range(n_in):
        # Get the current zonotope
        Z_Inf = Z_in_Inf[i]
        Z_Sup = Z_in_Sup[i]
        
        # Get the zonotope's parameters
        c_Inf = Z_Inf[0]
        c_Sup = Z_Sup[0]
        G_Inf = Z_Inf[1]
        G_Sup = Z_Sup[1]
        
        # Do the linear transformation
        c_Inf, c_Sup = intmat_mul(W_Inf, W_Sup, c_Inf, c_Sup)
        c_Inf = c_Inf + b
        c_Sup = c_Sup + b
        G_Inf, G_Sup = intmat_mul(W_Inf, W_Sup, G_Inf, G_Sup)
        
        # Update the output
        Z_out_Inf.append([c_Inf, G_Inf, Z_Inf[2], Z_Inf[3]])
        Z_out_Sup.append([c_Sup, G_Sup, Z_Sup[2], Z_Sup[3]])
    
    return Z_out_Inf, Z_out_Sup

In [ ]:
def hpint_perm(n):
    """
    This function outputs lists of matrices for calculating all permutations of half-plane
    intersections in the uncertain_ReLU_con_zono_single function.
    """
    c_new = []
    D_new = []
    H_new = []
    for i in range(2 ** n - 1):
        c_new_i = np.zeros((n, 1))
        binStr = bin(i + 1)[2:]
        for j in range(len(binStr)):
            c_new_i[n - 1 - j][0] = int(binStr[len(binStr) - 1 - j])
        c_new.append(c_new_i)
        D_new_i = np.diag(np.transpose(c_new_i)[0])
        D_new.append(D_new_i)
        H_new_i = np.diag(np.transpose(c_new_i * (-2) + 1)[0])
        H_new.append(H_new_i)
        
    return c_new, D_new, H_new

In [ ]:
def uncertain_abs(A_in_Inf, A_in_Sup):
    """
    The interval-matrix equivalent for the regular matrix operation abs(A). Takes in the
    infimum and supremum of an interval matrix A_in_Inf and A_in_Sup and returns the infimum
    and supremum matrices that contain the absolute value of each element in A_in.
    """
    # Preallocate the output
    A_out_Inf = np.zeros((A_in_Inf.shape))
    A_out_Sup = np.zeros((A_in_Inf.shape))
    
    # Perform operation element-by-element
    for i in range(A_in_Inf.shape[0]):
        for j in range(A_in_Inf.shape[1]):
            # If the infimum and supremum of the interval have different signs, the minimum
            # of their absolute value is 0. Otherwise, their minimum is simply the smaller of
            # the abosolute value of the bounds
            if not (A_in_Inf[i][j] <= 0. and A_in_Sup[i][j] >= 0.):
                A_out_Inf[i][j] = min([abs(A_in_Inf[i][j]), abs(A_in_Sup[i][j])])
            A_out_Sup[i][j] = max([abs(A_in_Inf[i][j]), abs(A_in_Sup[i][j])])
    
    return A_out_Inf, A_out_Sup

In [ ]:
def uncertain_ReLU_con_zono_single(Z_in_Inf, Z_in_Sup):
    """
    This function maps an uncertain constrained zonotope with infimum Z_in_Inf and supremum
    Z_in_Sup through a ReLU nonlinearity.
    
    Z_in_Inf and Z_in_Sup are lists with length of 4 as in [c, G, A, b], where c is the
    center, G is the generator matrix, (A, b) is the linear constraint array, of a constrained
    zonotope.
    
    The outputs are two lists with length n_out where each element is a zonotope. One list
    represents the infimum and the other list represents supremum.
    """
    ## Setup
    # Get the zonotope parameters
    c_Inf = Z_in_Inf[0]
    c_Sup = Z_in_Sup[0]
    G_Inf = Z_in_Inf[1]
    G_Sup = Z_in_Sup[1]
    A_Inf = Z_in_Inf[2]
    A_Sup = Z_in_Sup[2]
    b_Inf = Z_in_Inf[3]
    b_Sup = Z_in_Sup[3]
    
    # Get dimension of things
    n = c_Inf.shape[0]
    n_gen = G_Inf.shape[1]
    if type(A_Inf) == np.ndarray:
        n_con = A_Inf.shape[0]
    else:
        n_con = 0
    n_out_max = (2 ** n) - 1
    
    # Create list of output zonotopes
    Z_out_Inf = []
    Z_out_Sup = []
    
    ## Create the zonotopes
    c_new, D_new, H_new = hpint_perm(n)
    
    for i in range(n_out_max):
        # Get new center and generator matrices
        c_i_Inf, c_i_Sup = intmat_mul(D_new[i], D_new[i], c_Inf, c_Sup)
        G_i_Inf, G_i_Sup = intmat_mul(D_new[i], D_new[i], G_Inf, G_Sup)
        G_i_Inf = np.concatenate((G_i_Inf, np.zeros((n, n))), axis=1)
        G_i_Sup = np.concatenate((G_i_Sup, np.zeros((n, n))), axis=1)
        
        # Get new constraint arrays
        HG_Inf, HG_Sup = intmat_mul(H_new[i], H_new[i], G_Inf, G_Sup)
        d_i_Inf, d_i_Sup = uncertain_abs(HG_Inf, HG_Sup)
        d_i_Inf, d_i_Sup = intmat_mul(d_i_Inf, d_i_Sup, np.ones((n_gen, 1)), np.ones((n_gen, 1)))
        Hc_Inf, Hc_Sup = intmat_mul(H_new[i], H_new[i], c_Inf, c_Sup)
        d_i_Inf = 0.5 * (d_i_Inf - Hc_Sup)
        d_i_Sup = 0.5 * (d_i_Sup - Hc_Inf)
        
        b_i_Inf = -Hc_Sup - d_i_Sup
        b_i_Sup = -Hc_Inf - d_i_Inf
        if type(b_Inf) == np.ndarray:
            b_i_Inf = np.concatenate((b_Inf, b_i_Inf), axis=0)
            b_i_Sup = np.concatenate((b_Sup, b_i_Sup), axis=0)
            
        A_i_Inf = np.concatenate((HG_Inf, np.diag(np.transpose(d_i_Inf)[0])), axis=1)
        A_i_Sup = np.concatenate((HG_Sup, np.diag(np.transpose(d_i_Sup)[0])), axis=1)
        if n_con > 0:
            A_i_Inf = np.concatenate((np.concatenate((A_Inf, np.zeros((n_con, n))), axis=1), A_i_Inf), axis=0)
            A_i_Sup = np.concatenate((np.concatenate((A_Sup, np.zeros((n_con, n))), axis=1), A_i_Sup), axis=0)
        
        # Create output zonotope
        Z_out_Inf.append([c_i_Inf, G_i_Inf, A_i_Inf, b_i_Inf])
        Z_out_Sup.append([c_i_Sup, G_i_Sup, A_i_Sup, b_i_Sup])
        
    return Z_out_Inf, Z_out_Sup

In [ ]:
def uncertain_ReLU_con_zono(Z_in_Inf, Z_in_Sup):
    """
    This function maps a set of uncertain input zonotopes Z \in Z_in, Z = [c, G, A, b] through
    a ReLU nonlinearity.
    
    The inputs are two lists of constrained zonotopes with length n_in where each element is 
    [c, G, A, b], c is the center, G is the generator matrix, and linear constraint arrays (A,
    b), of a constrained zonotope. Z_in_Inf represents the infimum of the uncertain 
    constrained zonotopes, while Z_in_Sup represents the supremum.
    
    The outputs are similarly two lists of constrained zonotopes but with length n_out.
    """
    # Get the number of input zonotopes
    n_in = len(Z_in_Inf)
    
    # Iterate through input zonotopes and generate output
    Z_out_Inf = []
    Z_out_Sup = []
    
    for i in range(n_in):
        Z_i_Inf = Z_in_Inf[i]
        Z_i_Sup = Z_in_Sup[i]
        Z_out_i_Inf, Z_out_i_Sup = uncertain_ReLU_con_zono_single(Z_i_Inf, Z_i_Sup)
        for j in range(len(Z_out_i_Inf)):
            Z_out_Inf.append(Z_out_i_Inf[j])
            Z_out_Sup.append(Z_out_i_Sup[j])
            
    return Z_out_Inf, Z_out_Sup

In [ ]:
def forward_pass_uncertain_NN_con_zono(Z_in, NN_weights_Inf, NN_weights_Sup, NN_biases):
    """
    Compute a forward pass through the neural network with uncertain weights of infimum 
    NN_weights_Inf and supremum NN_weights_Sup, biases NN_biases, and ReLU activations, where
    the input is a single constrained zonotope [c, G, A, b].
    """
    # Get depth of neural network
    n_depth = len(NN_weights_Inf)
    
    # Run through layers and perform ReLU activations
    Z_out_Inf = [Z_in]
    Z_out_Sup = [Z_in]
    for i in range(n_depth - 1):
        W_Inf = NN_weights_Inf[i]
        W_Sup = NN_weights_Sup[i]
        b = NN_biases[i]
        Z_out_Inf, Z_out_Sup = uncertain_linear_layer_con_zono(Z_out_Inf, Z_out_Sup, W_Inf, W_Sup, b)
        Z_out_Inf, Z_out_Sup = uncertain_ReLU_con_zono(Z_out_Inf, Z_out_Sup)
        
    # Evaluate final layer
    Z_out_Inf, Z_out_Sup = uncertain_linear_layer_con_zono(Z_out_Inf, Z_out_Sup, NN_weights_Inf[n_depth - 1], NN_weights_Sup[n_depth - 1], NN_biases[n_depth - 1])

    return Z_out_Inf, Z_out_Sup
    

In [ ]:
## User parameters
# Random number generator seed
np.random.seed(0)

# Initial condition zonotope
c_0 = np.zeros((2, 1))
G_0 = 3. * np.eye(2)
A_0 = None
b_0 = None

# ReLU network parameters
NN_spec = [3]  # Each entry is the width of a layer, and the number of layers is the network depth
dim_in = 2  # Input dimension size
dim_out = 2  # Output dimension size
wt_lo = -1.  # Lower bound for random weights
wt_hi = 1.  # Upper bound for random weights

In [ ]:
## Automate from here
# Generate neural network
NN_weights_Inf, NN_weights_Sup, NN_biases = make_uncertain_ReLU_NN(NN_spec, dim_in, dim_out, wt_lo, wt_hi)

# Flow forward zonotope through the neural network
Z_in = [c_0, G_0, A_0, b_0]
Z_out_Inf, Z_out_Sup = forward_pass_uncertain_NN_con_zono(Z_in, NN_weights_Inf, NN_weights_Sup, NN_biases)

In [ ]:
# ## Testing area
# """
# Validate the outputs against those from the MATLAB script relu_intervalmat.m with rng(0) and
# NN_spec = [3]. Note that the order of the output uncertain zonotopes may be different.

# Result: Success!
# """
# NN_weights_Inf = [np.array([[-0.8147, -0.9134], [-0.9058, -0.6324], [-0.1270, -0.0975]]), np.array([[-0.9649, -0.9706, -0.4854], [-0.1576, -0.9572, -0.8003]])]
# NN_weights_Sup = [np.array([[0.8147, 0.9134], [0.9058, 0.6324], [0.1270, 0.0975]]), np.array([[0.9649, 0.9706, 0.4854], [0.1576, 0.9572, 0.8003]])]
# NN_biases = [np.array([[-0.4430], [0.0938], [0.9150]]), np.array([[-0.7162], [-0.1565]])]
# c_0 = np.zeros((2, 1))
# G_0 = 3. * np.eye(2)
# A_0 = None
# b_0 = None
# Z_in = [c_0, G_0, A_0, b_0]
# Z_out_Inf, Z_out_Sup = forward_pass_uncertain_NN_con_zono(Z_in, NN_weights_Inf, NN_weights_Sup, NN_biases)
# print(Z_out_Inf[0])
# print(Z_out_Sup[0])